In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
file_path = r'C:\Users\Lenovo\Documents\Work\Pelindo\data\Data Payment Praya\W5\PRAYA_LOG_0110_0710.xlsx'
df = pd.read_excel(file_path, sheet_name=['TPM', 'MNP', 'AMQ', 'BIT', 'BLW', 'T009', 'PNK', 'PLM', 'PJG'])

In [3]:
df['TPM'].head()

,INVOICE_NUMBER,NO_INTEGRATION_SAP,DOC_NO_SAP,PAYMENT_CODE,PAYMENT_DATE_IHUB,FLAGGING_PRAYA,STATUS,SAP_SEND_LOG,SAP_COMPLETE,FAKTUR_SAP
0,010.183.24-74.044257,I480141664/2024,9800140342/2024-3000,12410731140385,2024-10-07 15:20:00,2024-10-07 23:20:49,PAID,2024-10-07 23:21:07,2024-10-07 23:22:11,010.30186.24.00051495
1,010.183.24-74.044256,I480141663/2024,9800140341/2024-3000,12410731140377,2024-10-07 15:20:00,2024-10-07 23:20:48,PAID,2024-10-07 23:21:03,2024-10-07 23:21:09,010.30186.24.00051494
2,010.183.24-73.070310,I480141655/2024,9800140333/2024-3000,12410731141201,2024-10-07 14:56:00,2024-10-07 22:56:29,PAID,2024-10-07 22:57:02,2024-10-07 22:57:06,010.30186.24.00051492
3,010.183.24-73.070309,I480141653/2024,9800140331/2024-3000,12410731141182,2024-10-07 14:50:00,2024-10-07 22:50:48,PAID,2024-10-07 22:51:03,2024-10-07 22:51:07,010.30186.24.00051491
4,010.183.24-73.070307,I480141631/2024,9800140309/2024-3000,12410731141141,2024-10-07 14:05:00,2024-10-07 22:05:47,PAID,2024-10-07 22:06:03,2024-10-07 22:06:07,010.30186.24.00051489


In [4]:
for sheet in df:
    if sheet in ['TPM', 'MNP', 'BLW']:
        df[sheet]['GMT_PAYMENT_DATE_IHUB'] = df[sheet]['PAYMENT_DATE_IHUB'] + pd.Timedelta(hours=8)
    elif sheet == 'AMQ':
        df[sheet]['GMT_PAYMENT_DATE_IHUB'] = df[sheet]['PAYMENT_DATE_IHUB'] + pd.Timedelta(hours=9)
    elif sheet == 'BIT':
        df[sheet]['GMT_PAYMENT_DATE_IHUB'] = df[sheet]['PAYMENT_DATE_IHUB'] + pd.Timedelta(hours=7)

In [5]:
for sheet in df:
    if 'GMT_PAYMENT_DATE_IHUB' in df[sheet].columns:
        df[sheet]['FLAGGING_PRAYA_DELAY'] = df[sheet]['FLAGGING_PRAYA'] - df[sheet]['GMT_PAYMENT_DATE_IHUB']
        df[sheet]['SAP_SEND_DELAY'] = df[sheet]['SAP_SEND_LOG'] - df[sheet]['FLAGGING_PRAYA']
        df[sheet]['SAP_NOFAKTUR_GENERATE_TIME'] = df[sheet]['SAP_COMPLETE'] - df[sheet]['SAP_SEND_LOG']
        df[sheet]['TOTAL_TIME_TO_PRINT_INVOICE'] = df[sheet]['SAP_COMPLETE'] - df[sheet]['GMT_PAYMENT_DATE_IHUB']
    else:
        df[sheet]['FLAGGING_PRAYA_DELAY'] = df[sheet]['FLAGGING_PRAYA'] - df[sheet]['PAYMENT_DATE_IHUB']
        df[sheet]['SAP_SEND_DELAY'] = df[sheet]['SAP_SEND_LOG'] - df[sheet]['FLAGGING_PRAYA']
        df[sheet]['SAP_NOFAKTUR_GENERATE_TIME'] = df[sheet]['SAP_COMPLETE'] - df[sheet]['SAP_SEND_LOG']
        df[sheet]['TOTAL_TIME_TO_PRINT_INVOICE'] = df[sheet]['SAP_COMPLETE'] - df[sheet]['PAYMENT_DATE_IHUB']

In [6]:
# check if there are minus or negative value in FLAGGING_PRAYA_DELAY, SAP_SEND_DELAY, SAP_NOFAKTUR_GENERATE_TIME, TOTAL_TIME_TO_PRINT_INVOICE
for sheet in df:
    for col in ['FLAGGING_PRAYA_DELAY', 'SAP_SEND_DELAY', 'SAP_NOFAKTUR_GENERATE_TIME', 'TOTAL_TIME_TO_PRINT_INVOICE']:
        print(f'{sheet} - {col} : {df[sheet][df[sheet][col] < pd.Timedelta(0)][col].count()}')
    print('---------------------------------')

TPM - FLAGGING_PRAYA_DELAY : 2
TPM - SAP_SEND_DELAY : 2
TPM - SAP_NOFAKTUR_GENERATE_TIME : 0
TPM - TOTAL_TIME_TO_PRINT_INVOICE : 2
---------------------------------
MNP - FLAGGING_PRAYA_DELAY : 5
MNP - SAP_SEND_DELAY : 0
MNP - SAP_NOFAKTUR_GENERATE_TIME : 1
MNP - TOTAL_TIME_TO_PRINT_INVOICE : 4
---------------------------------
AMQ - FLAGGING_PRAYA_DELAY : 2
AMQ - SAP_SEND_DELAY : 0
AMQ - SAP_NOFAKTUR_GENERATE_TIME : 0
AMQ - TOTAL_TIME_TO_PRINT_INVOICE : 1
---------------------------------
BIT - FLAGGING_PRAYA_DELAY : 0
BIT - SAP_SEND_DELAY : 0
BIT - SAP_NOFAKTUR_GENERATE_TIME : 0
BIT - TOTAL_TIME_TO_PRINT_INVOICE : 0
---------------------------------
BLW - FLAGGING_PRAYA_DELAY : 2191
BLW - SAP_SEND_DELAY : 0
BLW - SAP_NOFAKTUR_GENERATE_TIME : 0
BLW - TOTAL_TIME_TO_PRINT_INVOICE : 2191
---------------------------------
T009 - FLAGGING_PRAYA_DELAY : 0
T009 - SAP_SEND_DELAY : 825
T009 - SAP_NOFAKTUR_GENERATE_TIME : 559
T009 - TOTAL_TIME_TO_PRINT_INVOICE : 818
----------------------------

In [17]:
# remove the minus or negative value in FLAGGING_PRAYA_DELAY or SAP_SEND_DELAY or SAP_NOFAKTUR_GENERATE_TIME or TOTAL_TIME_TO_PRINT_INVOICE 
df_filter = {}

for sheet in df:
    df_filter[sheet] = df[sheet].copy()
    for col in ['FLAGGING_PRAYA_DELAY', 'SAP_SEND_DELAY', 'SAP_NOFAKTUR_GENERATE_TIME', 'TOTAL_TIME_TO_PRINT_INVOICE']:
        df_filter[sheet] = df_filter[sheet][df_filter[sheet][col] >= pd.Timedelta(0)]
    # print total rows before and after filtering
    print(f'{sheet}: {df[sheet].shape[0]} -> {df_filter[sheet].shape[0]}')

TPM: 3140 -> 3136
MNP: 2359 -> 2353
AMQ: 986 -> 984
BIT: 1216 -> 1216
BLW: 2191 -> 0
T009: 825 -> 0
PNK: 1623 -> 0
PLM: 226 -> 0
PJG: 1288 -> 0


In [18]:
for sheet in df_filter:
    if 'GMT_PAYMENT_DATE_IHUB' in df_filter[sheet].columns:
        columns = list(df_filter[sheet].columns)
        columns.remove('GMT_PAYMENT_DATE_IHUB')
        columns.insert(5, 'GMT_PAYMENT_DATE_IHUB')
        df_filter[sheet] = df_filter[sheet][columns]

In [19]:
df_filter['TPM'].head()

,INVOICE_NUMBER,NO_INTEGRATION_SAP,DOC_NO_SAP,PAYMENT_CODE,PAYMENT_DATE_IHUB,GMT_PAYMENT_DATE_IHUB,FLAGGING_PRAYA,STATUS,SAP_SEND_LOG,SAP_COMPLETE,FAKTUR_SAP,FLAGGING_PRAYA_DELAY,SAP_SEND_DELAY,SAP_NOFAKTUR_GENERATE_TIME,TOTAL_TIME_TO_PRINT_INVOICE
0,010.183.24-74.044257,I480141664/2024,9800140342/2024-3000,12410731140385,2024-10-07 15:20:00,2024-10-07 23:20:00,2024-10-07 23:20:49,PAID,2024-10-07 23:21:07,2024-10-07 23:22:11,010.30186.24.00051495,0 days 00:00:49,0 days 00:00:18,0 days 00:01:04,0 days 00:02:11
1,010.183.24-74.044256,I480141663/2024,9800140341/2024-3000,12410731140377,2024-10-07 15:20:00,2024-10-07 23:20:00,2024-10-07 23:20:48,PAID,2024-10-07 23:21:03,2024-10-07 23:21:09,010.30186.24.00051494,0 days 00:00:48,0 days 00:00:15,0 days 00:00:06,0 days 00:01:09
2,010.183.24-73.070310,I480141655/2024,9800140333/2024-3000,12410731141201,2024-10-07 14:56:00,2024-10-07 22:56:00,2024-10-07 22:56:29,PAID,2024-10-07 22:57:02,2024-10-07 22:57:06,010.30186.24.00051492,0 days 00:00:29,0 days 00:00:33,0 days 00:00:04,0 days 00:01:06
3,010.183.24-73.070309,I480141653/2024,9800140331/2024-3000,12410731141182,2024-10-07 14:50:00,2024-10-07 22:50:00,2024-10-07 22:50:48,PAID,2024-10-07 22:51:03,2024-10-07 22:51:07,010.30186.24.00051491,0 days 00:00:48,0 days 00:00:15,0 days 00:00:04,0 days 00:01:07
4,010.183.24-73.070307,I480141631/2024,9800140309/2024-3000,12410731141141,2024-10-07 14:05:00,2024-10-07 22:05:00,2024-10-07 22:05:47,PAID,2024-10-07 22:06:03,2024-10-07 22:06:07,010.30186.24.00051489,0 days 00:00:47,0 days 00:00:16,0 days 00:00:04,0 days 00:01:07


In [20]:
# export to excel for each sheets
with pd.ExcelWriter(r'C:\Users\Lenovo\Documents\Work\Pelindo\data\Data Payment Praya\W5\PRAYA_LOG_0110_0710_v2.xlsx') as writer:
    for sheet in df:
        df_filter[sheet].to_excel(writer, sheet_name=sheet, index=False)

In [24]:
# add week (=5 Oktober) and cabang (sesuaikan dengan nama sheets) column in each sheet
df_v2 =  {}
for sheet in df:
    df_v2[sheet] = df_filter[sheet].copy()
    df_v2[sheet]['BULAN'] = 'OKTOBER'
    df_v2[sheet]['WEEK'] = '1'
    df_v2[sheet]['CABANG'] = sheet

In [25]:
# set WEEK as 1st column and CABANG as 2nd column
for sheet in df:
    columns = list(df_v2[sheet].columns)
    columns.remove('BULAN')
    columns.remove('WEEK')
    columns.remove('CABANG')
    columns.insert(0, 'BULAN')
    columns.insert(1, 'WEEK')
    columns.insert(2, 'CABANG')
    df_v2[sheet] = df_v2[sheet][columns]

In [26]:
df_v2['TPM'].head()

,BULAN,WEEK,CABANG,INVOICE_NUMBER,NO_INTEGRATION_SAP,DOC_NO_SAP,PAYMENT_CODE,PAYMENT_DATE_IHUB,GMT_PAYMENT_DATE_IHUB,FLAGGING_PRAYA,STATUS,SAP_SEND_LOG,SAP_COMPLETE,FAKTUR_SAP,FLAGGING_PRAYA_DELAY,SAP_SEND_DELAY,SAP_NOFAKTUR_GENERATE_TIME,TOTAL_TIME_TO_PRINT_INVOICE
0,OKTOBER,1,TPM,010.183.24-74.044257,I480141664/2024,9800140342/2024-3000,12410731140385,2024-10-07 15:20:00,2024-10-07 23:20:00,2024-10-07 23:20:49,PAID,2024-10-07 23:21:07,2024-10-07 23:22:11,010.30186.24.00051495,0 days 00:00:49,0 days 00:00:18,0 days 00:01:04,0 days 00:02:11
1,OKTOBER,1,TPM,010.183.24-74.044256,I480141663/2024,9800140341/2024-3000,12410731140377,2024-10-07 15:20:00,2024-10-07 23:20:00,2024-10-07 23:20:48,PAID,2024-10-07 23:21:03,2024-10-07 23:21:09,010.30186.24.00051494,0 days 00:00:48,0 days 00:00:15,0 days 00:00:06,0 days 00:01:09
2,OKTOBER,1,TPM,010.183.24-73.070310,I480141655/2024,9800140333/2024-3000,12410731141201,2024-10-07 14:56:00,2024-10-07 22:56:00,2024-10-07 22:56:29,PAID,2024-10-07 22:57:02,2024-10-07 22:57:06,010.30186.24.00051492,0 days 00:00:29,0 days 00:00:33,0 days 00:00:04,0 days 00:01:06
3,OKTOBER,1,TPM,010.183.24-73.070309,I480141653/2024,9800140331/2024-3000,12410731141182,2024-10-07 14:50:00,2024-10-07 22:50:00,2024-10-07 22:50:48,PAID,2024-10-07 22:51:03,2024-10-07 22:51:07,010.30186.24.00051491,0 days 00:00:48,0 days 00:00:15,0 days 00:00:04,0 days 00:01:07
4,OKTOBER,1,TPM,010.183.24-73.070307,I480141631/2024,9800140309/2024-3000,12410731141141,2024-10-07 14:05:00,2024-10-07 22:05:00,2024-10-07 22:05:47,PAID,2024-10-07 22:06:03,2024-10-07 22:06:07,010.30186.24.00051489,0 days 00:00:47,0 days 00:00:16,0 days 00:00:04,0 days 00:01:07


In [27]:
# Combine all sheets into a single DataFrame
combined_df = pd.DataFrame()

for sheet_name, data in df_v2.items():
    if 'GMT_PAYMENT_DATE_IHUB' in data.columns:
        data['PAYMENT_DATE_IHUB'] = data['GMT_PAYMENT_DATE_IHUB']
        data = data.drop(columns=['GMT_PAYMENT_DATE_IHUB'])
    combined_df = pd.concat([combined_df, data], ignore_index=True)

In [28]:
combined_df.head()

,BULAN,WEEK,CABANG,INVOICE_NUMBER,NO_INTEGRATION_SAP,DOC_NO_SAP,PAYMENT_CODE,PAYMENT_DATE_IHUB,FLAGGING_PRAYA,STATUS,SAP_SEND_LOG,SAP_COMPLETE,FAKTUR_SAP,FLAGGING_PRAYA_DELAY,SAP_SEND_DELAY,SAP_NOFAKTUR_GENERATE_TIME,TOTAL_TIME_TO_PRINT_INVOICE
0,OKTOBER,1,TPM,010.183.24-74.044257,I480141664/2024,9800140342/2024-3000,12410731140385,2024-10-07 23:20:00,2024-10-07 23:20:49,PAID,2024-10-07 23:21:07,2024-10-07 23:22:11,010.30186.24.00051495,0 days 00:00:49,0 days 00:00:18,0 days 00:01:04,0 days 00:02:11
1,OKTOBER,1,TPM,010.183.24-74.044256,I480141663/2024,9800140341/2024-3000,12410731140377,2024-10-07 23:20:00,2024-10-07 23:20:48,PAID,2024-10-07 23:21:03,2024-10-07 23:21:09,010.30186.24.00051494,0 days 00:00:48,0 days 00:00:15,0 days 00:00:06,0 days 00:01:09
2,OKTOBER,1,TPM,010.183.24-73.070310,I480141655/2024,9800140333/2024-3000,12410731141201,2024-10-07 22:56:00,2024-10-07 22:56:29,PAID,2024-10-07 22:57:02,2024-10-07 22:57:06,010.30186.24.00051492,0 days 00:00:29,0 days 00:00:33,0 days 00:00:04,0 days 00:01:06
3,OKTOBER,1,TPM,010.183.24-73.070309,I480141653/2024,9800140331/2024-3000,12410731141182,2024-10-07 22:50:00,2024-10-07 22:50:48,PAID,2024-10-07 22:51:03,2024-10-07 22:51:07,010.30186.24.00051491,0 days 00:00:48,0 days 00:00:15,0 days 00:00:04,0 days 00:01:07
4,OKTOBER,1,TPM,010.183.24-73.070307,I480141631/2024,9800140309/2024-3000,12410731141141,2024-10-07 22:05:00,2024-10-07 22:05:47,PAID,2024-10-07 22:06:03,2024-10-07 22:06:07,010.30186.24.00051489,0 days 00:00:47,0 days 00:00:16,0 days 00:00:04,0 days 00:01:07


In [29]:
# Export the combined DataFrame to a single sheet in an Excel file
combined_df.to_excel(r'C:\Users\Lenovo\Documents\Work\Pelindo\data\Data Payment Praya\W5\PRAYA_LOG_0110_0710_combined.xlsx', index=False)